In [6]:
import os
import pandas as pd

# Path to the folder where all the log files are stored
log_path = "/Feature_Extraction_from_VirtualBox_logs/VirtualBox_log_samples/"

final_data = pd.DataFrame() #Declaring an empty dataframe

count_feature = 0

for file in os.listdir(log_path):
    log_dict = {}
    count = 0
    os.chdir(log_path)
    with open(file, "r") as f:
        for line in f:
            line = line.split()
            log_dict[count] = line
            count += 1

    #Dropping an unwanted row
    for i in range(len(log_dict)):
        if len(log_dict[i]) > 2:
            if log_dict[i][2] == "Statistics":
                break
        log_dict.pop(i)

    new_list = []
    for s in log_dict.values():
        if s[2] == "Statistics" or s[2] == "End" or s[2] == "Detached" or s[1] == "NAT:":
            pass  #Skipping some unwanted rows
        elif s[2] == 'Max' and s[3] == 'TX' and s[4] == 'Delay':
            new_list.append(s)
            break #Stopping loop if reached end of line
        else:
            new_list.append(s)
    for i in range(len(new_list)):
        del new_list[i][0]

    df = pd.DataFrame()
    df['features'] = [i for i in range(len(new_list))]
    df['value'] = [i for i in range(len(new_list))]
    df['unit'] = [i for i in range(len(new_list))]

    list_1 = ["/Devices/8237A/DmaRun", "/Devices/hda/Stream0/Reset", "/Devices/hda/Stream0/Start", "/Devices/hda/Stream0/Stop", "/Devices/hda/Stream4/Reset", "/Devices/hda/Stream4/Start", "/Devices/hda/Stream4/Stop", "/Devices/vga/VMSVGA/Cmd/3dBlitSurfaceToScreenProf", "/Devices/vga/VMSVGA/EmtDelayOnBusyFifo", "/Devices/vga/VMSVGA/FifoExtendedSleep", "/Devices/vga/VMSVGA/FifoStalls", "/EM/CPU0/ExitOpt/Exec", "/EM/CPU0/ExitOpt/Probe", "/HGCM/MsgArrival", "/HGCM/MsgCompletion", "/HGCM/MsgTotal", "/IOM/MmioMappingsStale", "/PGM/Pool/Grow", "/PGM/ShMod/Check", "/PROF/CPU0/EM/Capped", "/PROF/CPU0/EM/NEMExec", "/PROF/CPU0/EM/Total", "/PROF/CPU0/VM/Halt/Block", "/PROF/CPU0/VM/Halt/BlockInsomnia", "/PROF/CPU0/VM/Halt/BlockOnTime", "/PROF/CPU0/VM/Halt/BlockOverslept", "/PROF/CPU0/VM/Halt/R0HaltBlock", "/PROF/CPU0/VM/Halt/R0HaltBlockInsomnia", "/PROF/CPU0/VM/Halt/R0HaltBlockOnTime", "/PROF/CPU0/VM/Halt/R0HaltBlockOverslept", "/PROF/CPU0/VM/Halt/Timers", "/PROF/CPU0/VM/Halt/Yield", "/TM/PIT/Handler"]

    def dataset_preprocess_1(data_frame):
        for i in range(len(data_frame)):
            if new_list[i][0] == "E1000#0:":
                fea = ""
                for k in new_list[i][1:-1]:
                    fea = fea + "_" + k
                data_frame['features'][i] = fea
                data_frame['value'][i] = new_list[i][-1]
                data_frame['unit'][i] = None
            elif new_list[i][0] == "/PDM/CritSects/TM" or new_list[i][0] == "/PDM/Tasks/001-HDA":
                data_frame['features'][i] = new_list[i][0] + '_' + new_list[i][1] + '_' + new_list[i][2]
                data_frame['value'][i] = new_list[i][3]
                data_frame['unit'][i] = new_list[i][4]
            elif new_list[i][0] == '/PDM/CritSectsRw/IOM':
                data_frame['features'][i] = new_list[i][0] + '_' + new_list[i][1]
                data_frame['value'][i] = new_list[i][2]
                data_frame['unit'][i] = new_list[i][3]
            elif len(new_list[i]) == 2:
                data_frame['features'][i] = new_list[i][0]
                data_frame['value'][i] = new_list[i][1]
                data_frame['unit'][i] = None
            elif len(new_list[i]) == 3:
                data_frame['features'][i] = new_list[i][0]
                data_frame['value'][i] = new_list[i][1]
                data_frame['unit'][i] = new_list[i][2]
            elif len(new_list[i]) == 4:
                data_frame['features'][i] = new_list[i][0] + '_' + new_list[i][1]
                data_frame['value'][i] = new_list[i][2]
                data_frame['unit'][i] = new_list[i][3]
            else:
                for k in list_1:
                    if new_list[i][0] == k:
                        data_frame['features'][i] = new_list[i][0]
                        data_frame['value'][i] = new_list[i][1]
                        data_frame['unit'][i] = new_list[i][2]

    dataset_preprocess_1(df)

    df = df[0:620] #Taking the first 620 columns
    df = df.transpose() #Converting rows to columns

    #Adding filename column for identification purpose
    filename = [None, None, None]
    df.insert(0, "filename", filename)
    df['filename'][1] = file

    if count_feature == 0:
        final_data = final_data.append(df.loc['features'])
        final_data = final_data.append(df.loc['value'])
    else:
        final_data = final_data.append(df.loc['value'])

    count_feature += 1

# The path to the folder where the final excel file will be saved
final_path = "/Feature_Extraction_from_VirtualBox_logs/"
final_data.to_excel(final_path + "final_dataset.xlsx")

C:\Users\Public\Documents\iSkysoft\CreatorTemp/ipykernel_6440/487163318.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['value'][i] = new_list[i][1]
C:\Users\Public\Documents\iSkysoft\CreatorTemp/ipykernel_6440/487163318.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['unit'][i] = new_list[i][2]
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i